In [61]:
import os
import pandas as pd
import geopandas as gpd
from collections import defaultdict
import csv

full_county_map_file = "../data/mappings/Complete_Counties_in_2022_Assembly.csv"
results_dir = "../data/2020_Primary_03"
statewide_results_file = os.path.join(results_dir, "Statewide", "statewide_sov_by_county.csv")

county_mappings = {r['County']: r['District'] for r in csv.DictReader(open(full_county_map_file))}
statewide_results = pd.read_csv(statewide_results_file, encoding='ISO-8859-1')

In [3]:
results = statewide_results[statewide_results.CONTEST_NAME == 'President Democratic'][statewide_results.COUNTY_NAME != 'State Totals'] #TODO
total_results = defaultdict(dict)

for n,v in results.groupby(["COUNTY_NAME", 'CANDIDATE_NAME']):
    total_results[n[0]][n[1]] = int(v['VOTE_TOTAL'])

/var/folders/3l/n7z238yj4cz86pdyffy5_5hc0000gq/T/ipykernel_3676/80036635.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = statewide_results[statewide_results.CONTEST_NAME == 'President Democratic'][statewide_results.COUNTY_NAME != 'State Totals'] #TODO


In [150]:
district_results = {dist_num+1: defaultdict(int) for dist_num in range(80)}
for county_name, district in county_mappings.items():
    county_results = total_results[county_name]
    for candidate, votes in county_results.items():
        district_results[int(district)][candidate] += votes

In [154]:
dirname = "../data/2020_Primary_03/reshaped"
for fname in os.listdir(dirname):
    print(fname)
    if "swp" not in fname:
        with open(os.path.join(dirname, fname)) as f:
            for row in csv.DictReader(f):
                dist = int(row['district'])
                for key in row:
                    if key != "district":
                        district_results[dist][key] += int(row[key])

San_Francisco_County.csv
San_Mateo_County.csv
Orange_County.csv
Los_Angeles_County.csv
.Los_Angeles_County.csv.swp


In [155]:
outfile = f"../data/2020_Primary_03/Statewide/Statewide_By_2022_Assembly.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in district_results.items():
    results['district'] = district
    data = {k: results.get(k,0) for k in write_cols}
    print(district, data)
    writer.writerow(data)

1 {'district': 1, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
2 {'district': 2, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
3 {'district': 3, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
4 {'district': 4, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
5 {'district': 5, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
6 {'district': 6, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
7 {'district': 7, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
8 {'district': 8, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
9 {'district': 9, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH WARREN': 0, 'BERNIE SANDERS': 0}
10 {'district': 10, 'BALLOTS CAST': 0, 'JOSEPH R. BIDEN': 0, 'ELIZABETH W

In [55]:
# Orange County
county = "Orange"
oc_results_file = f"../data/2020_Primary_03/{county}_county/{county}_County.csv"
oc_res = pd.read_csv(oc_results_file, skiprows=1, encoding='ISO-8859-1', dtype={'Precinct': str})

oc_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
oc_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(oc_mapping_file))}

county_results = defaultdict(lambda: defaultdict(int))
for (precinct, candidate), votes in  oc_res[oc_res['Contest Party'] == 'DEM'][oc_res['Contest Title'] == 'PRESIDENT OF THE UNITED STATES'].groupby(['Precinct', 'Choice Name']):
    district = int(oc_mappings[precinct])
    if 'write-ins' not in candidate:
        count = int(votes['Total Votes'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count
            
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    print(district, data)
    writer.writerow(data)

In [34]:
county = "San_Francisco"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
#county_res = pd.read_csv(county_results_file, encoding='ISO-8859-1', dtype={'Precinct': str})
county_res = list(csv.DictReader(open(county_results_file, encoding='ISO-8859-1')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
#county_res

In [ ]:
cleaned_results = {}
last_prec = None
for row in county_res:
    if 'Pct' in row['ï»¿Electionwide']:
        last_prec = row['ï»¿Electionwide']
    elif row['ï»¿Electionwide'] == 'Total':
        cleaned_results[last_prec] = row
        last_prec = None

county_results = defaultdict(lambda: defaultdict(int))
for prec, results in cleaned_results.items():
    if prec is None:
        continue
    precinct = prec.split(' ')[1]
    if precinct not in county_mappings:
        precinct = precinct.split('/')[0]
    if precinct in county_mappings:
        district = county_mappings[precinct]
        for key, val in results.items():
            if '(DEM)' in key:
                candidate = key.replace(" (DEM) ", "")
                county_results[district][candidate] += int(val)
            elif key == 'Total Votes':
                county_results[district]["BALLOTS CAST"] += int(val)

In [92]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [122]:
county = "San_Mateo"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
county_res = pd.read_csv(county_results_file, dtype={'Precinct_name': str})
#county_res = list(csv.DictReader(open(county_results_file, encoding='ISO-8859-1')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res

,Precinct_name,Split_name,Reporting_flag,Update_count,Pct_Id,Pct_seq_nbr,Reg_voters,Turn_Out,Contest_Id,Contest_seq_nbr,...,total_under_votes,total_over_votes,Vote Centers_ballots,Vote Centers_votes,Vote Centers_under_votes,Vote Centers_over_votes,Vote by Mail_ballots,Vote by Mail_votes,Vote by Mail_under_votes,Vote by Mail_over_votes
0,1001,NaN,1,0,1,1,3313,0.67,1,11,...,20,1,293,0,3,0,1269,1,17,1
1,1001,NaN,1,0,1,1,3313,0.67,1,11,...,20,1,293,0,3,0,1269,0,17,1
2,1001,NaN,1,0,1,1,3313,0.67,1,11,...,20,1,293,44,3,0,1269,197,17,1
3,1001,NaN,1,0,1,1,3313,0.67,1,11,...,20,1,293,0,3,0,1269,1,17,1
4,1001,NaN,1,0,1,1,3313,0.67,1,11,...,20,1,293,1,3,0,1269,13,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33633,5301,5301-1,1,0,387,387,111,0.58,11,28,...,10,0,5,3,1,0,59,42,9,0
33634,5301,5301-1,1,0,387,387,111,0.58,11,28,...,10,0,5,1,1,0,59,8,9,0
33635,5301,5301-1,1,0,387,387,111,0.58,22,33,...,17,0,5,4,1,0,59,43,16,0
33636,5301,5301-1,1,0,387,387,111,0.58,26,37,...,3,0,5,3,0,0,59,36,3,0


In [126]:
county_results = defaultdict(lambda: defaultdict(int))

for (precinct, candidate), votes in  county_res[county_res['Contest_title'] == 'President of the United States, Democratic'].groupby(['Precinct_name', 'candidate_name']):
    district = int(county_mappings[precinct])
    if 'write-ins' not in candidate:
        count = int(votes.sum()['total_votes'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count

In [127]:
       
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(
    open(outfile, "w", newline=""), write_cols
)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

In [51]:
county = "SLO"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County.csv"
county_res = pd.read_csv(county_results_file, dtype={'PRECINCT_NAME': str})
#county_res = list(csv.DictReader(open(county_results_file, encoding='ISO-8859-1')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res

,PRECINCT_NAME,PRECINCT_ID,REGISTER_VOTERS,BALLOT_CAST,CONTEST_FULL_NAME,CONTEST_ID,CONTEST_SEQ,VOTES_ALLOWED,CANDIDATE_FULL_NAME,CANDIDATE_ID,PARTY_CODE,VOTE_COUNT,REPORTING_FLAG
0,101,1,1972,311,President Of The United States-Democratic,1001,1,1,JULIÁN CASTRO,1,NaN,1,0
1,101,1,1972,311,President Of The United States-Democratic,1001,1,1,MICHAEL A. ELLINGER,2,NaN,0,0
2,101,1,1972,311,President Of The United States-Democratic,1001,1,1,ELIZABETH WARREN,3,NaN,28,0
3,101,1,1972,311,President Of The United States-Democratic,1001,1,1,MARIANNE WILLIAMSON,4,NaN,2,0
4,101,1,1972,311,President Of The United States-Democratic,1001,1,1,ANDREW YANG,5,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10867,MB06,142,0,0,Member of the State Assembly 37th District,1250,12,1,STEVE BENNETT,69,NaN,0,0
10868,MB06,142,0,0,Member of the State Assembly 37th District,1250,12,1,JONATHAN ABBOUD,70,1.0,0,0
10869,MB06,142,0,0,Member of the State Assembly 37th District,1250,12,1,Write-in,101,NaN,0,0
10870,MB06,142,0,0,PROP 13-School Bonds,9010,16,1,YES,89,NaN,0,0


In [53]:
county_results = defaultdict(lambda: defaultdict(int))
grouped = county_res[county_res['CONTEST_FULL_NAME'] == 'President Of The United States-Democratic'].groupby(['PRECINCT_NAME', 'CANDIDATE_FULL_NAME'])
nonstandard_precincts_SLO = {
    'MB01': 'M1',
    'MB02': 'M2',
    'MB03': 'M3',
    'MB04': 'M4',
    'MB05': 'M5',
}
for (precinct, candidate), votes in grouped:
    if precinct not in county_mappings:
        if precinct in nonstandard_precincts_SLO:
            district = int(county_mappings[nonstandard_precincts_SLO[precinct]])
        else:
            # should be ok since MB06 doesn't have any registered voters
            continue
    else:
        district = int(county_mappings[precinct])
        
    if 'write-ins' not in candidate:
        count = int(votes.sum()['VOTE_COUNT'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count

In [54]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)

71

In [64]:
county = "Alameda"
county_results_file = f"../data/2020_Primary_03/{county}_County/{county}_County_sov.csv"
county_res = pd.read_csv(county_results_file, dtype={'PRECINCT_NAME': str})
#county_res = list(csv.DictReader(open(county_results_file, encoding='ISO-8859-1')))

county_mapping_file = f"../data/mappings/{county}_County_March_2020_as_2022_Assembly.csv"
county_mappings = {r['precinct']: r['District'] for r in csv.DictReader(open(county_mapping_file))}
county_res

,PRECINCT_NAME,PRECINCT_ID,REGISTER_VOTERS,BALLOT_CAST,CONTEST_FULL_NAME,CONTEST_ID,CONTEST_SEQ,VOTES_ALLOWED,CANDIDATE_FULL_NAME,CANDIDATE_ID,PARTY_CODE,VOTE_COUNT,REPORTING_FLAG
0,200200,1,1279,5,AI - Presidential Preference,10,1,1,PHIL COLLINS,28,3.0,0,0
1,200200,1,1279,5,AI - Presidential Preference,10,1,1,"ROQUE ""ROCKY"" DE LA FUENTE",29,3.0,0,0
2,200200,1,1279,5,AI - Presidential Preference,10,1,1,CHARLES KRAUT,30,3.0,0,0
3,200200,1,1279,5,AI - Presidential Preference,10,1,1,J.R. MYERS,31,3.0,0,0
4,200200,1,1279,5,AI - Presidential Preference,10,1,1,DON BLANKENSHIP,32,3.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123883,9880500,1014,0,0,Measure C -Alameda County,470,47,1,NO,247,NaN,0,0
123884,9880500,1014,0,0,Bond Measure D - Alameda County Fire Department,480,48,1,YES,248,NaN,0,0
123885,9880500,1014,0,0,Bond Measure D - Alameda County Fire Department,480,48,1,NO,249,NaN,0,0
123886,9880500,1014,0,0,Bond Measure M - Pleasanton USD,440,58,1,BONDS YES,240,NaN,0,0


In [62]:
precs = gpd.read_file('../shapefiles/2020_Primary_03/Alameda_County/srprec_001_p20_v01.shp')
precs.head()

,SRPREC,COUNTY,geometry
0,200200,001,"POLYGON ((-122.24813 37.88115, -122.24835 37.8..."
1,201000,001,"POLYGON ((-122.26299 37.89542, -122.26299 37.8..."
2,201100,001,"POLYGON ((-122.26924 37.89651, -122.26928 37.8..."
3,201400,001,"POLYGON ((-122.26634 37.88379, -122.26635 37.8..."
4,201410,001,"POLYGON ((-122.25410 37.88910, -122.25410 37.8..."


In [80]:
county_results = defaultdict(lambda: defaultdict(int))
grouped = county_res[county_res['CONTEST_FULL_NAME'] == 'DEM - Presidential Preference'].groupby(['PRECINCT_NAME', 'CANDIDATE_FULL_NAME'])
for (precinct, candidate), votes in grouped:
    district = int(county_mappings[precinct])
        
    if 'write-ins' not in candidate:
        count = int(votes.sum()['VOTE_COUNT'])
        if count > 0:
            county_results[district][candidate] += count
            county_results[district]['BALLOTS CAST'] += count

defaultdict(<function __main__.<lambda>()>,
            {14: defaultdict(int,
                         {'AMY KLOBUCHAR': 1642,
                          'BALLOTS CAST': 81560,
                          'ANDREW YANG': 296,
                          'BERNIE SANDERS': 28523,
                          'CORY BOOKER': 47,
                          'ELIZABETH WARREN': 23864,
                          'JOSEPH R. BIDEN': 17808,
                          'MICHAEL BENNET': 12,
                          'MICHAEL R. BLOOMBERG': 6567,
                          'PETE BUTTIGIEG': 2154,
                          'TOM STEYER': 322,
                          'TULSI GABBARD': 205,
                          'DEVAL PATRICK': 4,
                          'JOE SESTAK': 7,
                          'JOHN K. DELANEY': 13,
                          'MICHAEL A. ELLINGER': 40,
                          'JULIÁN CASTRO': 10,
                          'MARIANNE WILLIAMSON': 27,
                          'MOSIE BOYD':

In [82]:
outfile = f"../data/2020_Primary_03/reshaped/{county}_County.csv"
results_cols = ['JOSEPH R. BIDEN', 'ELIZABETH WARREN', 'BERNIE SANDERS']
write_cols = ["district", 'BALLOTS CAST'] + results_cols
writer = csv.DictWriter(open(outfile, "w", newline=""), write_cols)
writer.writeheader()
for district, results in county_results.items():
    results['district'] = district
    data = {k: results[k] for k in write_cols}
    writer.writerow(data)